In [ ]:
import os
from joblib import Parallel, delayed
import time
import numpy as np
from collections import defaultdict


In [ ]:
sudo_password = ''

In [ ]:
with open('all_external_ips') as f:
    lines = f.read().splitlines()


lines = [x for x in lines if x!='None']
lines

for ip in lines:
    if ip!='None':
        current = (ip.split('.'))

        print('ssh -i "MAKWest.pem" ubuntu@ec2-'+str(current[0])+'-'\
              +str(current[1])+'-'+str(current[2])+'-'+str(current[3])+'.us-west-1.compute.amazonaws.com')

In [ ]:
nclusters = 0

for folder in os.listdir('.'):
    if 'config' in folder and folder!='config':
        nclusters = nclusters + 1
        
        
print(nclusters)


total_count = 8
nclients = 2


nreplicas = int(total_count/nclusters)
nodeIps = lines[:nclusters* nreplicas]


clientIps = lines[-nclients:]



In [ ]:
    
def compile_job(i):
    ip = lines[i]
    if ip!='None':
        current = (ip.split('.'))
        command = f'echo {sudo_password} | sudo -S ./gradlew installDist'
        
        
        os.system(command)

        
        
def clean_job(i):
    ip = lines[i]
    

    if ip!='None':
        current = (ip.split('.'))
        command = f'echo {sudo_password} | sudo -S rm ../dump/nohup*.out; echo 3108 | sudo -S rm config*/currentView;'
        
        
        os.system(command)
        
        

        
def run_server_node(i):
    ip = lines[i]
    if ip!='None' and( ip in nodeIps):
        
        current = (ip.split('.'))
        command = 'nohup ./runscripts/startReplicaYCSB.sh '+str(i)+' >../dump/nohup_s'+str(i)+'.out 2>&1 & echo $! > ../dump/jobID_s'+str(i)+'.txt'
        
        print(command)
        os.system(command)
        
        




def run_basic_client(i):
    cmd_arg = ''
    
    ip = lines[i]
    if ip!='None' and ip in clientIps:
        current = (ip.split('.'))
        for iter__ in range(len(clientIps)):
            if ip==clientIps[iter__]:
                cmd_arg = iter__
                
        current = (ip.split('.'))
        command = 'nohup ./runscripts/ycsbClient.sh '+str(i-len(nodeIps))+' >../dump/nohup_c'+str(i)+'.out 2>&1 &'

        
        os.system(command)


def kill_java(i):
    ip = lines[i]
    
    if ip!='None':
        current = (ip.split('.'))
        command = f'echo {sudo_password} | sudo -S killall java;echo 3108: sudo -S killall nohup'
        
        
        os.system(command)
        
        
        
def kill_clients(i):
    ip = lines[i]
    if ip!='None' and ip in clientIps:
        current = (ip.split('.'))
        command = 'killall java;  killall nohup"'
        
        
        os.system(command)




In [ ]:
len(lines)

In [ ]:
len(nodeIps), len(clientIps)

In [ ]:
for i in range(len(lines)):
    kill_java(i)

for i in range(len(lines)):
    clean_job(i)
    
compile_job(1)



In [ ]:
    
for i in [0,1,2,3,4,5,6,7]:
    print(i)
    run_server_node(i)

In [ ]:
time.sleep(10)
# !tail ../dump/nohup_s0.out -n 50

In [ ]:


# for i in range(len(lines)):
#     kill_java(i)

In [ ]:
len(nodeIps)+len(clientIps)


## Run basic client

In [ ]:
for i in range(len(nodeIps),len(nodeIps)+len(clientIps)):
    run_basic_client(i)

In [ ]:
time.sleep(80)

In [ ]:
for i in range(len(lines)):
    kill_java(i)

In [ ]:

def getTimesThputs(file):
    f = open(file, "r")
    data = f.readlines() 
    times = []
    thputs = []
    latencies = []
    for line in data:
#         print(line)
        if not line.startswith('--') and 'current ops/sec; ' in line and 'AverageLatency(us)=' in line and not 'JVM' in line:

#         print(line)
            times.append(line.split('sec:')[0].strip())
            thputs.append(line.split(';')[1].split('current')[0].strip())


            read_lat = '0'
            write_lat = '0'

            if ('READ AverageLatency' in line):

                print(line)

                read_lat = line.split('READ AverageLatency(us)=')[1].split(']')[0].strip()


            if ('UPDATE AverageLatency' in line):                    

                write_lat = line.split('UPDATE AverageLatency(us)=')[1].split(']')[0].strip()


            if (('READ AverageLatency' not in line) and ('UPDATE AverageLatency' not in line)):
                latencies.append(line.split('=')[1].split(']')[0].strip())
            else:
                read_frac = 0
                write_frac = 0

                if float(read_lat) > 0:
                    read_frac = 0.85

                if float(write_lat) > 0:
                    write_frac = 0.15

                lat = str( (write_frac * float(write_lat) +\
                            read_frac*float(read_lat))/(write_frac+read_frac)\
                         )
                latencies.append(lat)

            print(line,latencies[-1],thputs[-1],times[-1])

    return np.array(times).astype(float), np.array(thputs).astype(float), np.array(latencies).astype(float)/1e6  


In [ ]:
times ,thputs, lats = getTimesThputs('../dump/nohup_c'+str(len(nodeIps))+'.out')
times2 ,thputs2, lats = getTimesThputs('../dump/nohup_c'+str(len(nodeIps)+1)+'.out')




In [ ]:

def combine_throughputs(times1, thputs1, times2, thputs2):
    thput_dict = defaultdict(int)

    # Populate the dictionary with the first set
    for t, th in zip(times1, thputs1):
        thput_dict[t] += th

    # Populate the dictionary with the second set
    for t, th in zip(times2, thputs2):
        thput_dict[t] += th

    # Sort the keys (times) and extract corresponding values
    combined_times = sorted(thput_dict.keys())
    combined_thputs = [thput_dict[t] for t in combined_times]

    return combined_times, combined_thputs

In [ ]:
times_new, thputs_new = combine_throughputs(times, thputs, times2, thputs2)

    

times_new, thputs_new

In [ ]:
def fill_missing_times(times, throughputs):
    # Create a full range from min to max time
    full_times = list(range(min(times), max(times) + 1))
    
    # Create a dictionary of existing times and throughputs
    throughput_dict = dict(zip(times, throughputs))
    
    # Fill missing times with zero throughput
    filled_throughputs = [throughput_dict.get(t, 0) for t in full_times]
    
    return full_times, filled_throughputs

In [ ]:
times_new, thputs_new = fill_missing_times(np.array(times_new).astype(int), thputs_new)

In [ ]:
import pylab as pl

pl.figure(figsize = (12,8))
pl.plot(times_new[:], thputs_new[:], '-o', label='ava-bftsmart')
pl.xlabel('Time (s)')
pl.ylabel('Throughput (Txn/s)')
pl.legend()
pl.show()
pl.clf()
